#### 1. Obtención base de datos - Web Scraping

Este notebook incluye el código utilizado para scrapear la información de la url `https://www.theca.org.uk/orcas/reports`. En esta página web se incluyen los informes obtenidos de cuestionarios que se formularon a skippers que experienciaron una interacción con orcas entre marzo del 2022 y noviembre del 2023. Tras obtener la información en el orden deseado, se pasa a un csv que será tratado en un notebook aparte.

In [1]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
from selenium.webdriver.common.by import By
from IPython.display import Image # Para mostrar imágenes en python
from bs4 import BeautifulSoup as bs
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

pd.set_option('display.max_columns', None)

In [2]:
# Creamos y abrimos un driver 
from selenium import webdriver
PATH = 'driver/chromedriver.exe'
driver = webdriver.Chrome()

Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)
Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)


In [3]:
# Definimos URL de búsqueda
url = 'https://www.theca.org.uk/orcas/reports'

In [4]:
driver.get(url)

In [5]:
driver.find_element(By.XPATH, '/html/body/div[5]/div/div/div[2]/button[2]').click() # Clickamos en "No cookies, thanks"

In [6]:
# Voy a contar la cantidad de casos que hay para luego saber cuanto buclear
time.sleep(1)
casos_text = driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[1]/div/div[1]').text
casos_number = int(casos_text.split('(')[1].split(')')[0])
casos_number


155

In [7]:
# Clickamos en "SHOW LIST"
time.sleep(2)
driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[1]/strong').click()

In [8]:
# Clickamos en primera pestaña con datos del caso más reciente
time.sleep(1)
driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[4]/div[1]/p').click()


In [9]:
# Voy a comprobar a ver si puedo buclear por las tabs de cada interacción:

# Cerramos caso actual
time.sleep(1)
driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[1]/span[1]').click()


* Estamos en la posicion que queremos. El reto ahora es conseguir hacer un bucle que recorra las distintas tabs con los datos de cada interacción orca-barco.

* Hacemos un primer bucle de tres pasos como prueba. 

La secuencia es la siguiente:
1) Clickamos sobre la pestaña del caso en que nos encontremos
2) Simulamos lectura de la info
3) cerramos pestaña y repetimos bucle

In [10]:
# Como ya hemos calculado previamente cuantos casos de interaccion tenemos realmente, ya sé cuando va a estar bucleando
# y leyendo respuestas del cuestionario

# Pruebo a buclear a traves de todas las tabs, hacemos un primer intento con 3 vueltas
for i in range(1, 4, 1):
    time.sleep(1)
    driver.find_element(By.XPATH, f'/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[4]/div[{i}]/p').click()
    print('Acabo de abrir tab')
    time.sleep(1)
    print('Simulo que estoy leyendo info y cierro tab')
    time.sleep(1)
    driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[1]/span[1]').click()


Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab


* Ya sé como iterar a través de todas las tabs, solo habría que cambiar el segundo elemento del range(a, b, c) por el cantidad de interacciones que haya en la web, valor que ya se calcula automáticamente previamente.

* Ahora vamos a pasar a pensar como ir pasando cada lista a un database y de ahí a un csv que será trabajado más adelante.

In [11]:
# Defino en primer lugar el dataframe con las columnas de interés:
columnas = ["date", "lat_and_long", "GTOA_Protocol", "Boat_Type", "Boat_Length", "Towing_Inflatable", "Trailing_Fishing_Lure",
           "Physical_Contact_With_Boat", "Number_of_Adult_Orcas", "Number_of_Juvenile_Orcas", "Number_of_Uncertain_Age_Orcas", "Rudder", "Motoring_or_Sailing",
           "Speed_Knots", "Sea_State", "Wind_Speed_Beaufort", "Daylight_or_Darkness", "Cloud_Cover",
           "Distance_Off_Land_NM", "Depth_Meters", "Depth_Gauge", "Autopilot", "Hull_Topsides_Color",
           "Antifoul_Color", "Boat_Damaged", "Tow_Required", "Crew_Response", "Orcas_Behaviour"]

df = pd.DataFrame(columns=columnas)
df.head()

,date,lat_and_long,GTOA_Protocol,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour


* Llegados a este punto llegamos a la problemática de recoger los datos, y meterlos en un dataframe

* Un pregunta que afecta a la estructura de las preguntas y respuestas es la cantidad y edad de orcas que se observaron en la interacción. La pregunta número 7 para los skippers es cuantas orcas adultas, cuantas juveniles y/o cuantas de edad desconocida fueron vistas en la interacción. El problema viene cuando se dejan en blanco alguna de la casilla por lo que jugamos con las siguientes opciones:
1) Adultas | NO Juveniles | NO Desconocidas
2) Adultas | Juveniles | NO Desconocidas
3) Adultas | NO Juveniles | Desconocidas
4) Adultas | Juveniles | Desconocidas
5) NO Adultas | Juveniles | NO Desconocidas
6) NO Adultas | Juveniles | Desconocidas
7) NO Adultas | NO Juveniles | Desconocidas
8) NO Adultas | NO Juveniles | NO Desconocidas -> Tipo de formulario distinto a partir del reporte 138

Va a tocar hacer los 8 casos y para ello se hará uso de varios if - elif() y posiblemente de una llamada a una función previamente definida.

In [12]:
# Caso 1: Adultas | NO Juveniles | NO Desconocidas

def lista_caso1(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(1)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(0.5)
    # Appendeo pregunta 5: Trailing fishing lure?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 6: Did orcas make physical contact with boat
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(1)
    # Appendeo pregunta 7: Number of adult orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(1)
    # Appendeo Juveniles vacía
    lista_resps.append(0)
    time.sleep(0.5)
    # Appendeo Uncertain vacía
    lista_resps.append(0)
    time.sleep(0.5)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[49]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[51]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[53]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps

In [13]:
# Caso 2: Adultas | Juveniles | NO Desconocidas

def lista_caso2(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(1)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(0.5)
    # Appendeo pregunta 5: Trailing fishing lure?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 6: Did orcas make physical contact with boat
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(1)
    # Appendeo pregunta 7: Number of adult orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(1)
    # Appendeo Number of Juvenile orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(1)
    # Appendeo Uncertain vacía
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[49]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[51]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[53]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[55]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps

In [14]:
# Caso 3: Adultas | NO Juveniles | Desconocidas
def lista_caso3(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(1)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(0.5)
    # Appendeo pregunta 5: Trailing fishing lure?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 6: Did orcas make physical contact with boat
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(1)
    # Appendeo pregunta 7: Number of adult orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(1)
    # Appendeo Juveniles vacía
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo Number of uncertain orcas
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(0.5)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[49]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[51]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[53]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[55]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps

In [15]:
# Caso 4: Adultas | Juveniles | Desconocidas

def lista_caso4(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(1)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(0.5)
    # Appendeo pregunta 5: Trailing fishing lure?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 6: Did orcas make physical contact with boat
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(1)
    # Appendeo pregunta 7: Number of adult orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(1)
    # Appendeo Number of Juvenile orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(1)
    # Appendeo Number of Uncertain orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(1)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[49]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[51]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[53]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[55]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[57]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps


In [16]:
# Caso 5: NO Adultas | Juveniles | NO Desconocidas

def lista_caso5(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(1)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(0.5)
    # Appendeo pregunta 5: Trailing fishing lure?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 6: Did orcas make physical contact with boat
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(0.5)
    # Appendeo Adultos vacía
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo Pregunta 7: Number of Juvenile orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(1)
    # Appendeo Uncertain vacía
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[49]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[51]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[53]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps

In [17]:
# Caso 6: NO Adultas | Juveniles | Desconocidas

def lista_caso6(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(1)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(0.5)
    # Appendeo pregunta 5: Trailing fishing lure?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 6: Did orcas make physical contact with boat
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(0.5)
    # Appendeo Adultos vacía
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo pregunta 7: Number of Juvenile orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(1)
    # Appendeo Number of uncertain orcas
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(1)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[49]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[51]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[53]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[55]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps

In [18]:
# Caso 7: NO Adultas | NO Juveniles | Desconocidas

def lista_caso7(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(1)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(0.5)
    # Appendeo pregunta 5: Trailing fishing lure?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 6: Did orcas make physical contact with boat
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(1)
    # Appendeo Adultos vacía
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo Juveniles vacía
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo pregunta 7: Number of Uncertain orcas seen?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(0.5)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[49]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[51]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[53]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps

In [19]:
# Caso 8: NO Adultas | NO Juveniles | NO Desconocidas
# Antes del informe 138 no se preguntaba cuantas orcas habían interactuado. Necesitamos una función que pase un 0 para las 3 columnas en cuestión.

def lista_caso8(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(0.5)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(1)
    # Appendeo pregunta 5: Trailing fishing lure? -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append('Unknown')
    time.sleep(1)
    # Appendeo pregunta 6: Did orcas make physical contact with boat -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append('Unknown')
    time.sleep(1)
    # Appendeo Adults vacía -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo Juveniles vacía -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo Uncertain vacía -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps

In [20]:
# Caso Alternativo: NO Adultas | NO Juveniles | NO Desconocidas
# Informe 15 tiene una estructura diferente.

def lista_alter(): 
    
    # Defino lista vacía
    lista_resps = []
    time.sleep(0.5)
    # Relleno lista con respuestas
    # Appendeo fecha
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
    time.sleep(0.5)
    # Appendeo lat. y long. juntos
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
    time.sleep(0.5)
    # Appendeo pregunta 1: GTOA Protocol
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
    time.sleep(0.5)
    # Appendeo pregunta 2: Boat Type
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
    time.sleep(0.5)
    # Appendeo pregunta 3: Boat length (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
    time.sleep(0.5)
    # Appendeo pregunta 4: Towing an inflatable?
    lista_resps.append('Unknown')
    time.sleep(1)
    # Appendeo pregunta 5: Trailing fishing lure? -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append('Unknown')
    time.sleep(1)
    # Appendeo pregunta 6: Did orcas make physical contact with boat -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append('Unknown')
    time.sleep(1)
    # Appendeo Adults vacía -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo Juveniles vacía -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append(0)
    time.sleep(1)
    # Appendeo Uncertain vacía -> Esta pregunta no se hacía antes del informe 138
    lista_resps.append(4)
    time.sleep(1)
    # Appendeo pregunta 8: Rudder type
    lista_resps.append('Unknown')
    time.sleep(0.5)
    # Appendeo pregunta 9: Motoring or sailing?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
    time.sleep(0.5)
    # Appendeo pregunta 10: Speed (knots)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
    time.sleep(0.5)
    # Appendeo pregunta 11: Sea state
    lista_resps.append('Unknown')
    time.sleep(0.5)
    # Appendeo pregunta 12: Wind Speed (Beaufort)
    lista_resps.append('Unknown')
    time.sleep(0.5)
    # Appendeo pregunta 13: Darkness or daylight
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
    time.sleep(0.5)
    # Appendeo pregunta 14: Cloud cover
    lista_resps.append('Unknown')
    time.sleep(0.5)
    # Appendeo pregunta 15: Distance off land (nautical miles)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
    time.sleep(0.5)
    # Appendeo pregunta 16: Depth (metres)
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
    time.sleep(0.5)
    # Appendeo pregunta 17:Depth gauge ON?
    lista_resps.append('Unknown')
    time.sleep(0.5)
    # Appendeo pregunta 18: Autopilot ON?
    lista_resps.append('Unknown')
    time.sleep(0.5)
    # Appendeo pregunta 19: Hull topside colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
    time.sleep(0.5)
    # Appendeo pregunta 20: Antifoul colour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
    time.sleep(0.5)
    # Appendeo pregunta 21: Was the boat damaged and in need of a repair?
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
    time.sleep(0.5)
    # Appendeo pregunta 22: Tow required
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
    time.sleep(0.5)
    # Appendeo pregunta 23: Crew response
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
    time.sleep(0.5)
    # Appendeo pregunta 24: Orca behaviour
    lista_resps.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
    time.sleep(0.5)
    time.sleep(1)
    # En este punto tengo la lista con todas las respuestas hasta orca behaviour

    return lista_resps

* Llegados a este punto ya hemos definido una función para cada posibilidad, estas funciones nos devolverán una lista con los datos correspondientes a cada fila de información del dataframe. Cada pregunta formulada es una columna del dataframe.

* Ahora vamos a pasar a separar los distintos casos con ifs-elifs() dentro del bucle. La idea es comparar strings de las preguntas (7), (8), y (9) y llamar a la función que corresponda.

* Dentro del bucle se seguirá la siguiente secuencia:
1) Abrir pestaña del caso en cuestión
2) Recorrer if-elif() para comprobar en que caso estamos
3) Llamar a la función que corresponda al caso en cuestión
4) Añadir lista devuelta por la función al dataframe
5) Cerrar tab y bucle recomienza

In [22]:
for i in range(1, casos_number, 1):
    
    # Abro tab
    time.sleep(0.5)
    driver.find_element(By.XPATH, f'/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[4]/div[{i}]/p').click()
    print('Acabo de abrir tab')

    # Voy a aplicar un webdriver para esperar hasta que se haya abierto la tab, para evitar errores
    element = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[1]/span[1]')))

    try: 
        # Voy a definir variables que contengan la string de las preguntas 7, 8 y 9:
        time.sleep(0.5)
        q7 = driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[18]').text
        q8 = driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[20]').text
        q9 = driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[22]').text

        # Defino también variables con la pregunta tipo para no enmarronar:
        Adultas = "Number of adult orcas seen"
        Juveniles = "Number of juvenile orcas seen"
        Uncertain = "Number of orcas, uncertain of age, seen"
        Distance = "Distance off land (nautical miles)"        

        # Comienzo a diferenciar los casos:
        time.sleep(0.5)
        if (q7 == Adultas) & (q8 != Juveniles) & (q9 != Uncertain): # Caso: Adultas | NO Juveniles | NO Desconocidas
            lst = lista_caso1()
        elif (q7 == Adultas) & (q8 == Juveniles) & (q9 != Uncertain): # Caso: Adultas | Juveniles | NO Desconocidas
            lst = lista_caso2()
        elif (q7 == Adultas) & (q8 != Juveniles) & (q9 == Uncertain): # Caso: Adultas | NO Juveniles | Desconocidas
            lst = lista_caso3()
        elif (q7 == Adultas) & (q8 == Juveniles) & (q9 == Uncertain): # Caso: Adultas | Juveniles | Desconocidas
            lst = lista_caso4()
        elif (q7 == Juveniles) & (q8 != Uncertain): # Caso: NO Adultas | Juveniles | NO Desconocidas
            lst = lista_caso5()
        elif (q7 == Juveniles) & (q8 == Uncertain): # Caso: NO Adultas | Juveniles | Desconocidas
            lst = lista_caso6()
        elif (q7 == Uncertain): # Caso: NO Adultas | NO Juveniles | Desconocidas
            lst = lista_caso7()
        elif (q7 == Distance):
            lst = lista_alter()
        else:
            lst = lista_caso8()

        # Meto la lista en el dataframe
        df.loc[i-1] = lst

    except:
        unknown_list = ['Unknown' for _ in range(28)]
        df.loc[i-1] = unknown_list

    print('Simulo que estoy leyendo info y cierro tab')

    # Paso a cerrar la tab y pasar al siguiente paso cuando recomience bucle
    driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[1]/span[1]').click()


Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de

In [23]:
df.tail(10)

,date,lat_and_long,GTOA_Protocol,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour
144,2022-06-04 08:00:00,"36 4.4000 N, 5 59.9000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Spade,Unknown,Unknown,0,0,0,Motoring,3 - 4,0 - 2 (0 - 6 knots),Day,5 - 10,40 - 200m,On,On,White/light,Black,"Yes, minor - will wait until the end of the se...",No,This report was obtained by GTOA. I stopped th...,This report was obtained by GTOA. It was only ...,daytime,waxing\n20% illuminated,Within 3 days of spring tide
145,2022-06-01 12:30:00,"36 20.1000 N, 6 16.7400 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Unknown,Unknown,Unknown,0,0,4,Unknown,Twin rudder,Sailing,Unknown,Unknown,Day,Unknown,2 - 5,20 - 40m,Unknown,Unknown,On,On,White/light,Black,"Yes, extensive - major works required",Yes
146,2022-07-16 21:15:00,"43 1.4000 N, 9 32.8000 W","No, did not follow protocol, interaction laste...",Sail,Over 15m,Not towing,Unknown,Unknown,0,0,0,Spade,Motorsailing,5 - 7,Moderate,5 - 6 (17 - 27 knots),Dusk,75 - 100%,Over 10,200m+,On,On,Dark colour,Black,No,No,"The group of Orcas, we saw at least 2, probabl...",Very fast and excited approach from the stern ...
147,2022-07-12 04:05:00,"47 6.0000 N, 5 30.0000 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Not towing,Unknown,Unknown,0,0,0,Spade,Motorsailing,5 - 7,Moderate,3 - 4 (7 - 16 knots),Night,0 - 25%,Over 10,40 - 200m,On,Off,White/light,Coppercoat,No,No,At first i noticed the presence by blows and d...,before dawn
148,2022-07-06 07:15:00,"35 59.0000 N, 5 55.0000 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Not towing,Unknown,Unknown,0,0,0,Twin rudder,Motorsailing,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,5 - 10,40 - 200m,On,On,White/light,Black,No,No,We were following another sail boat from Porti...,"They appeared from my starboard, went under th..."
149,2022-07-03 18:30:00,"39 24.6240 N, 9 38.4150 W","Yes, followed protocol, interaction lasted 10 ...",Sail,10 - 12.5m,Not towing,Unknown,Unknown,0,0,0,Twin rudder,Sailing,5 - 7,Moderate,3 - 4 (7 - 16 knots),Day,50 - 75%,Over 10,200m+,On,On,White/light,Black,"Yes, extensive - major works required",No,Nous naviguions à la voile avec un catamaran d...,"Relativement calme, allant sous le bateau, don..."
150,2022-07-03 17:50:00,"38 7.2100 N, 9 1.4300 W","Yes, followed protocol, interaction lasted les...",Sail,Under 10m,Not towing,Unknown,Unknown,0,0,0,Spade,Motorsailing,5 - 7,Calm,5 - 6 (17 - 27 knots),Day,25 - 50%,5 - 10,40 - 200m,On,On,White/light,Blue,No,No,I only saw 2 adult females and 2 juveniles. My...,Very placid and almost lethargic. I’ve never s...
151,2022-07-03 13:00:00,"39 21.0000 N, 9 35.0000 W","No, did not follow protocol, interaction laste...",Sail,Over 15m,Not towing,Unknown,Unknown,0,0,0,Twin rudder,Motorsailing,5 - 7,Moderate,7+ (28 knots+),Day,25 - 50%,5 - 10,40 - 200m,On,Off,White/light,Black,"Yes, extensive - major works required",No,The boat is a catamaran with 2 engines and rud...,There were 3 adults and 2 juveniles. The small...
152,2022-06-24 12:50:00,"36 50.0000 N, 8 55.0000 W","No, did not follow protocol, interaction laste...",Sail,10 - 12.5m,Not towing,Unknown,Unknown,0,0,0,Spade,Sailing,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,Over 10,200m+,On,On,White/light,Blue,No,No,The two orcas were nearby a buoy and as soon a...,The two orcas (one of 6 meters and other of 3-...
153,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,U

In [24]:
df.shape

(154, 28)

In [25]:
# Pasamos el dataframe a nuestra carpeta data en formato csv:
df.to_csv('../data/scraped_data.csv', index=False)

In [26]:
# Cerramos el driver
driver.quit()

1.WebScraping - done